## Decoupler

In [1]:
# !cd /raid1/cqy/CD45-/Graduation_project

In [2]:
import numpy as np
import pandas as pd
import anndata as an
import matplotlib.pyplot as plt
import seaborn as sns
import decoupler
import os
from scipy.sparse import csr_matrix

In [2]:
# GSEA via msigDB
msigdb = decoupler.get_resource("MSigDB")

/home/cqy/anaconda3/envs/decoupler/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
450kB [00:00, 890kB/s]
  File "/home/cqy/anaconda3/envs/decoupler/lib/python3.12/site-packages/urllib3/response.py", line 737, in _error_catcher
    yield
  File "/home/cqy/anaconda3/envs/decoupler/lib/python3.12/site-packages/urllib3/response.py", line 1187, in read_chunked
    chunk = self._handle_chunk(amt)
            ^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/cqy/anaconda3/envs/decoupler/lib/python3.12/site-packages/urllib3/response.py", line 1129, in _handle_chunk
    value = self._fp._safe_read(amt)  # type: ignore[union-attr]
            ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/cqy/anaconda3/envs/decoupler/lib/python3.12/http/client.py", line 642, in _safe_read
    raise IncompleteRead(data, amt-len(data))
http.client.In

In [3]:
hm = msigdb[msigdb["collection"] == "hallmark"]
hm = hm[~hm.duplicated(("geneset", "genesymbol"))]

### 1. Alv. fibro.

In [2]:
msigdb = pd.read_csv("msigdb.csv", index_col = 0)
msigdb

,genesymbol,collection,geneset
0,MAFF,chemical_and_genetic_perturbations,BOYAULT_LIVER_CANCER_SUBCLASS_G56_DN
1,MAFF,chemical_and_genetic_perturbations,ELVIDGE_HYPOXIA_UP
2,MAFF,chemical_and_genetic_perturbations,NUYTTEN_NIPP1_TARGETS_DN
3,MAFF,immunesigdb,GSE17721_POLYIC_VS_GARDIQUIMOD_4H_BMDC_DN
4,MAFF,chemical_and_genetic_perturbations,SCHAEFFER_PROSTATE_DEVELOPMENT_12HR_UP
...,...,...,...
5522261,COMPLEX:CD79A_CD79B,go_biological_process,GOBP_POSITIVE_REGULATION_OF_IMMUNE_RESPONSE
5522262,COMPLEX:CD79A_CD79B,human_phenotype_ontology,HP_UNUSUAL_INFECTION_BY_ANATOMICAL_SITE
5522263,COMPLEX:CD79A_CD79B,go_biological_process,GOBP_REGULATION_OF_IMMUNE_RESPONSE
5522264,COMPLEX:CD79A_CD79B,cell_type_signatures,DESCARTES_FETAL_INTESTINE_LYMPHOID_CELLS


In [42]:
markers = pd.read_csv("3.Stroma/alv_marker.csv")
markers = markers.iloc[:, 4:8]
markers.columns = ["names", "pvals", "logfc", "scores"]
markers = markers[markers["pvals"] < 0.05]
markers = markers.sort_values("logfc", ascending = False)[["names", "logfc"]].set_index("names")
markers

,logfc
names,
LIF,4.816750
PTX3,4.294945
IL6,3.667414
CXCL1,3.526358
CXCL2,3.453327
...,...
TMEM74B,-1.223328
ZNF33B,-1.267573
ING4,-1.296530


In [43]:
## hallmark
geneset_size = hm.groupby("geneset").size()
gsea_genesets= geneset_size.index[(geneset_size > 15) & (geneset_size < 500)] # Match behaviour of fgsea

scores, norm, pvals = decoupler.run_gsea(
    markers.T,
    hm[hm["geneset"].isin(gsea_genesets)],
    source="geneset",
    target="genesymbol",
)

gsea_results = pd.concat({"score": scores.T, "norm": norm.T, "pval": pvals.T}, axis = 1)
gsea_results = gsea_results.droplevel(level = 1, axis = 1).sort_values("norm", ascending = False)
gsea_results.head(20)

,score,norm,pval
source,,,
HALLMARK_TNFA_SIGNALING_VIA_NFKB,0.777629,2.283913,0.000000
HALLMARK_INFLAMMATORY_RESPONSE,0.720094,2.036541,0.000000
HALLMARK_MYC_TARGETS_V2,0.667393,1.815551,0.000000
HALLMARK_INTERFERON_GAMMA_RESPONSE,0.628013,1.793522,0.000000
HALLMARK_IL2_STAT5_SIGNALING,0.597373,1.719489,0.000000
HALLMARK_UV_RESPONSE_UP,0.606681,1.717827,0.000000
HALLMARK_IL6_JAK_STAT3_SIGNALING,0.638495,1.714815,0.000000
HALLMARK_ESTROGEN_RESPONSE_LATE,0.604074,1.691311,0.000000
HALLMARK_HYPOXIA,0.574130,1.642719,0.000000


In [44]:
fdrcorrection(pd.Series(gsea_results["pval"]).tolist())[1]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.03847634, 0.00278614, 0.01431434, 0.05465859,
       0.01001001, 0.00278614, 0.00526316, 0.06813627, 0.23267839,
       0.02045454, 0.27815115, 0.32583369, 0.330136  , 0.330136  ,
       0.330136  , 0.39439621, 0.36875   , 0.41070484, 0.4814922 ,
       0.58750001, 0.7066003 , 0.71341461, 0.7066003 , 0.71341461,
       0.7066003 , 0.73799803, 0.73799803, 0.78874334, 0.78874334,
       0.78874334, 0.87738538, 1.        , 1.        , 1.        ])

In [45]:
from statsmodels.stats.multitest import fdrcorrection
gsea_results["padj"] = fdrcorrection(pd.Series(gsea_results["pval"]).tolist())[1]
gsea_results.to_csv("3.Stroma/alv_gsea.csv")
gsea_results

,score,norm,pval,padj
source,,,,
HALLMARK_TNFA_SIGNALING_VIA_NFKB,0.777629,2.283913,0.000000,0.000000
HALLMARK_INFLAMMATORY_RESPONSE,0.720094,2.036541,0.000000,0.000000
HALLMARK_MYC_TARGETS_V2,0.667393,1.815551,0.000000,0.000000
HALLMARK_INTERFERON_GAMMA_RESPONSE,0.628013,1.793522,0.000000,0.000000
HALLMARK_IL2_STAT5_SIGNALING,0.597373,1.719489,0.000000,0.000000
HALLMARK_UV_RESPONSE_UP,0.606681,1.717827,0.000000,0.000000
HALLMARK_IL6_JAK_STAT3_SIGNALING,0.638495,1.714815,0.000000,0.000000
HALLMARK_ESTROGEN_RESPONSE_LATE,0.604074,1.691311,0.000000,0.000000
HALLMARK_HYPOXIA,0.574130,1.642719,0.000000,0.000000


### 2. Myofibro.

In [32]:
markers = pd.read_csv("3.Stroma/myo_marker.csv")
markers = markers.iloc[:, 4:8]
markers.columns = ["names", "pvals", "logfc", "scores"]
markers = markers[markers["pvals"] < 0.05]
markers = markers.sort_values("logfc", ascending = False)[["names", "logfc"]].set_index("names")
markers

,logfc
names,
CCL19,4.162661
CXCL10,3.760087
CXCL11,3.650998
CXCL9,2.869823
TNFSF14,2.677394
...,...
COL10A1,-3.481847
DERL3,-3.522932
LUZP2,-4.098281


In [34]:
geneset_size = hm.groupby("geneset").size()
gsea_genesets= geneset_size.index[(geneset_size > 15) & (geneset_size < 500)] # Match behaviour of fgsea

scores, norm, pvals = decoupler.run_gsea(
    markers.T,
    hm[hm["geneset"].isin(gsea_genesets)],
    source="geneset",
    target="genesymbol"
)

gsea_results = pd.concat({"score": scores.T, "norm": norm.T, "pval": pvals.T}, axis = 1)
gsea_results = gsea_results.droplevel(level = 1, axis = 1).sort_values("norm", ascending = False)
gsea_results.head(10)

,score,norm,pval
source,,,
HALLMARK_INTERFERON_GAMMA_RESPONSE,0.700270,3.040198,0.000000
HALLMARK_INTERFERON_ALPHA_RESPONSE,0.685993,2.545914,0.000000
HALLMARK_TNFA_SIGNALING_VIA_NFKB,0.556497,2.492684,0.000000
HALLMARK_IL6_JAK_STAT3_SIGNALING,0.681572,2.388499,0.000000
HALLMARK_INFLAMMATORY_RESPONSE,0.586930,2.308730,0.000000
HALLMARK_XENOBIOTIC_METABOLISM,0.550573,2.253476,0.000000
HALLMARK_ALLOGRAFT_REJECTION,0.583372,2.229837,0.000000
HALLMARK_UV_RESPONSE_UP,0.593441,2.191411,0.000000
HALLMARK_COMPLEMENT,0.473073,1.808203,0.000000


In [35]:
from statsmodels.stats.multitest import fdrcorrection
gsea_results["padj"] = fdrcorrection(pd.Series(gsea_results["pval"]).tolist())[1]
gsea_results.to_csv("3.Stroma/myo_gsea.csv")
gsea_results

,score,norm,pval,padj
source,,,,
HALLMARK_INTERFERON_GAMMA_RESPONSE,0.700270,3.040198,0.000000,0.000000
HALLMARK_INTERFERON_ALPHA_RESPONSE,0.685993,2.545914,0.000000,0.000000
HALLMARK_TNFA_SIGNALING_VIA_NFKB,0.556497,2.492684,0.000000,0.000000
HALLMARK_IL6_JAK_STAT3_SIGNALING,0.681572,2.388499,0.000000,0.000000
HALLMARK_INFLAMMATORY_RESPONSE,0.586930,2.308730,0.000000,0.000000
HALLMARK_XENOBIOTIC_METABOLISM,0.550573,2.253476,0.000000,0.000000
HALLMARK_ALLOGRAFT_REJECTION,0.583372,2.229837,0.000000,0.000000
HALLMARK_UV_RESPONSE_UP,0.593441,2.191411,0.000000,0.000000
HALLMARK_COMPLEMENT,0.473073,1.808203,0.000000,0.000000


### 3. Fibroblast Pseudotime Analysis

In [12]:
fibro = an.read_h5ad("3.Stroma/fibro_diff.h5ad")
fibro

AnnData object with n_obs × n_vars = 58590 × 1000
    obs: 'method', 'sampleID', 'disease', 'dataset', 'sex', 'age', 'smoking_status', 'major_cell_type', 'cellID', 'histology', 'leiden', 'minor_cell_type', 'batch_variety', 'leiden_R', 'leiden_R2', 'diff_cluster', 'dpt_pseudotime'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'dataset_colors', 'diff_cluster_colors', 'diffmap_evals', 'disease_colors', 'hvg', 'iroot', 'leiden', 'leiden_R2_colors', 'minor_cell_type_colors', 'minor_cell_type_sizes', 'neighbors', 'paga', 'pca', 'rank_genes_groups', 'umap'
    obsm: 'X_diffmap', 'X_pca', 'X_pca_harmony', 'X_umap'
    obsp: 'connectivities', 'distances'

In [13]:
ct = pd.read_csv("3.Stroma/pseudotime_gene_cluster.csv")
ct.columns = ["genesymbol", "cluster"]
ct.head(5)

,genesymbol,cluster
0,ANKRD36C,1
1,B4GALT6,1
2,TNNT2,1
3,CARMN,1
4,COL13A1,1


In [14]:
decoupler.run_aucell(fibro, ct,
                     source = "cluster",
                     target = "genesymbol",
                     use_raw = True)

In [16]:
pathway_score = fibro.obsm["aucell_estimate"].copy()
pathway_score.to_csv("3.Stroma/pseudotime_aucell.csv")

### 3. Endo

In [9]:
endo = an.read_h5ad("4.Endo/Endo.h5ad")
endo

AnnData object with n_obs × n_vars = 80050 × 1000
    obs: 'method', 'sampleID', 'disease', 'dataset', 'sex', 'age', 'smoking_status', 'major_cell_type', 'cellID', 'histology', 'leiden', 'minor_cell_type', 'batch_variety', 'leiden_R'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'dataset_colors', 'disease_colors', 'hvg', 'leiden', 'leiden_R_colors', 'leiden_colors', 'minor_cell_type_colors', 'neighbors', 'pca', 'rank_genes_groups', 'umap'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [42]:
endo_geneset = ["GOBP_LEUKOCYTE_ADHESION_TO_VASCULAR_ENDOTHELIAL_CELL",
                "BIOCARTA_VEGF_PATHWAY",
                "GOBP_ANTIGEN_PROCESSING_AND_PRESENTATION",
                "REACTOME_COLLAGEN_FORMATION",
                "REACTOME_INTERFERON_SIGNALING"]
endo_geneset = msigdb[msigdb["geneset"].isin(endo_geneset)]
endo_geneset = endo_geneset[~endo_geneset.duplicated(("geneset", "genesymbol"))]

In [43]:
decoupler.run_aucell(endo, endo_geneset,
                     source = "geneset",
                     target = "genesymbol",
                     use_raw = True)

In [44]:
acts = decoupler.get_acts(endo, obsm_key='aucell_estimate')
acts.write_h5ad("4.Endo/Endo_aucell.h5ad")
acts

AnnData object with n_obs × n_vars = 80050 × 5
    obs: 'method', 'sampleID', 'disease', 'dataset', 'sex', 'age', 'smoking_status', 'major_cell_type', 'cellID', 'histology', 'leiden', 'minor_cell_type', 'batch_variety', 'leiden_R'
    uns: 'dataset_colors', 'disease_colors', 'hvg', 'leiden', 'leiden_R_colors', 'leiden_colors', 'minor_cell_type_colors', 'neighbors', 'pca', 'rank_genes_groups', 'umap'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap', 'aucell_estimate'

In [45]:
pathway_score = endo.obsm["aucell_estimate"].copy()
pathway_score["minor_cell_type"] = endo.obs["minor_cell_type"]
pathway_score.to_csv("4.Endo/aucell_pathway.csv")